In [ ]:
%run ./utility_functions.ipynb

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices = ["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"])

# Load Dataset

In [ ]:
image_shape = (128,128,3)

In [ ]:
train_dataset_path = "../Dataset/clean/TensorflowFaces/"
valid_dataset_path = "../Dataset/clean/TensorflowFaces_test/"

BATCH_SIZE_PER_REPLICA = 32
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

trainLoader = Loader(path=train_dataset_path,
                     image_shape=image_shape,
                     batch_size=BATCH_SIZE,
                     max_pos_pairs= 90000,
                     apply_augmentation=False,
                     augmentation_pourcentage=0.5,
                     preprocess_pipeline = None,
                     normalize = True,
                     image_per_celeb=15)

training_dataset = trainLoader.dataset

validLoader = Loader(path=valid_dataset_path,
                     image_shape=image_shape,
                     batch_size=BATCH_SIZE,
                     max_pos_pairs=5000,
                     preprocess_pipeline=None,
                     normalize=True)

validation_dataset = validLoader.dataset

In [ ]:
trainLoader.visualize(value_range=(0,1),
                      color_mode_switch=False)

# Training

## MobileNet

In [ ]:
log_dir = f"../runs/{datetime.now().strftime('%Y%m%d=%H%M%S')}"

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,        
    histogram_freq=1,         
    write_graph=False,         
    write_images=False         
)

activation_callb = actionCallback(training_dataset,validation_dataset, log_dir, BATCH_SIZE)

with strategy.scope(): 
    
    optimizer = Adam(learning_rate=1e-4)
    
    distance_fc = similarityDistances(normalize=True).cosine_distance
    loss_fc = contrastiveLoss(margin=1.0, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE, distance_fc=distance_fc, hard_mining=False)
    
    dist_sim_metric = Distance(name="sim_distance_metric", aggregation_method="quartil", mode=0, distance_fc = distance_fc)
    dist_dissim_metric = Distance(name="dissim_distance_metric", aggregation_method="quartil", mode=1, distance_fc = distance_fc)
    
    acc = Accuracy(threshold=0.5, name="accuraccy", distance_fc = distance_fc)
    
    feature_extractor = get_feature_extractor_2(l2reg=1e-4)
    

    s_network = siameseNetwork(input_size=image_shape, base_model=feature_extractor, embedding_dimension = 128, dropout=0.2, l2_reg=1e-4, name="siamese", apply_augmentation=False)
    print(s_network.base_model.summary())

    s_network.compile(optimizer=optimizer,
                      loss = loss_fc,
                      metrics = [dist_sim_metric, dist_dissim_metric, acc])
    
    s_network.fit(training_dataset, epochs=15, validation_data=validation_dataset, callbacks=[tensorboard_callback, activation_callb])

#  Evaluating

In [ ]:
with strategy.scope():
    
    optimizer = Adam(learning_rate=1e-3)
    loss_fc = contrastiveLoss(margin=1.0, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    dist_sim_metric = Distance(name="sim_distance_metric", aggregation_method="quartil", mode=0)
    dist_dissim_metric = Distance(name="dissim_distance_metric", aggregation_method="quartil", mode=1)
    
    acc = Accuracy(threshold=0.38, name="accuraccy")
    # acc = BinaryAccuracy(threshold = 0.5)
    
    s_network.compile(optimizer=optimizer,
                      loss = loss_fc,
                      metrics = [dist_sim_metric, dist_dissim_metric, acc])
    
    s_network.evaluate(validation_dataset)